In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/soCzech/TransNet

!pip install protobuf==3.20.*
!pip install ffmpeg-python

Cloning into 'TransNet'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 24
Receiving objects: 100% (27/27), 32.91 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
%cd /content/TransNet

!chmod +x transnet.py # Thêm vào khi chạy trên colab

# import tensorflow.compat.v1 as tf # dòng này thêm vào file transnet.py khi clone git của transnet về, thay vào vị trí của import tensorflow as tf
# tf.disable_v2_behavior() # dòng này thêm vào file transnet.py khi clone git của transnet về

/content/TransNet


In [ ]:
import ffmpeg
import numpy as np
import tensorflow as tf

from transnet import TransNetParams, TransNet
from transnet_utils import draw_video_with_predictions, scenes_from_predictions

import glob
import os
import cv2
import csv

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
params = TransNetParams()
params.CHECKPOINT_PATH = "./model/transnet_model-F16_L3_S2_D256"

net = TransNet(params)

[TransNet] Creating ops.
           Input (?, ?, 27, 48, 3)
           SDDCNN_1
           > DDCNN_1 (?, ?, 27, 48, 64)
           > DDCNN_2 (?, ?, 27, 48, 64)
           MaxPool (?, ?, 13, 24, 64)
           SDDCNN_2
           > DDCNN_1 (?, ?, 13, 24, 128)
           > DDCNN_2 (?, ?, 13, 24, 128)
           MaxPool (?, ?, 6, 12, 128)
           SDDCNN_3
           > DDCNN_1 (?, ?, 6, 12, 256)
           > DDCNN_2 (?, ?, 6, 12, 256)
           MaxPool (?, ?, 3, 6, 256)
           Flatten (?, ?, 4608)
           Dense (?, ?, 256)
           Logits (?, ?, 2)
           Predictions (?, ?)
[TransNet] Network built.
[TransNet] Found 4614850 trainable parameters.
[TransNet] Parameters restored from 'transnet_model-F16_L3_S2_D256'.


In [ ]:
################# Config Folder #################
video_paths = sorted(glob.glob('/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/Database-BTC/Videos_L02/video/*.mp4'))
des_path = '/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/KeyFrames_AIO_Pending/KeyFrames_L02_Optimize2/'
csv_des_folder = '/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/map-keyframes'
#################################################

In [ ]:

for video_path in video_paths:
    print(video_path)
    folder_name = video_path.split('/')[-1].replace( '.mp4','')

    # print(test_folder)
    # if test_folder <= 244 or test_folder==253 or test_folder==271:
    #     print(f"Skip {folder_name}")
    #     continue

    folder_path = des_path + f'{folder_name}'
    print(folder_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # export video into numpy array using ffmpeg
    print(video_path)
    video_stream, err = (
        ffmpeg
        .input(video_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(params.INPUT_WIDTH, params.INPUT_HEIGHT))
        .run(capture_stdout=True)
    )
    video = np.frombuffer(video_stream, np.uint8).reshape([-1, params.INPUT_HEIGHT, params.INPUT_WIDTH, 3])

    # predict transitions using the neural network
    predictions = net.predict_video(video)

    # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
    scenes = scenes_from_predictions(predictions, threshold=0.1)
    with open(f"{folder_path}.txt", 'w') as f:
        for sc in scenes:
            f.write(str(sc) + '\n')

    cam = cv2.VideoCapture(video_path)
    currentframe = 0
    index = 0

    # Create folder to save csv
    csv_des_path = csv_des_folder + f'/{folder_name}'
    if not os.path.exists(csv_des_folder):
      os.makedirs(csv_des_folder)
    n=1

    # Write csv
    with open(f"{csv_des_path}.csv", 'w', newline='') as csvfile:
      fieldnames = ['n','pts_time','fps','frame_idx']
      writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
      # writer.writerheader() # writeheaders is new in Python 2.7 / 3.2
      writer.writerow(dict((fn,fn) for fn in fieldnames)) # 1 cách writerheader() khác

      while True:
          ret,frame = cam.read()
          if ret:
              currentframe += 1
              # for sc in scenes:
              idx_first = int(scenes[index][0])
              idx_end = int(scenes[index][1])
              idx_025 = int(scenes[index][0] + (scenes[index][1]-scenes[index][0])/4)
              idx_05 = int(scenes[index][0] + (scenes[index][1]-scenes[index][0])/2)
              idx_075 = int(scenes[index][0] + 3*(scenes[index][1]-scenes[index][0])/4)

              #### First ####
              if currentframe - 1 == idx_first:
                  filename_first = "{}/{:0>6d}.jpg".format(folder_path, idx_first)
                  # video_save = cv2.resize(video[idx_first], (1280,720))
                  cv2.imwrite(filename_first, frame)

                  # SAVE CSV
                  writer.writerow({'n':n,'pts_time': (idx_first/25),'fps': 25,'frame_idx': idx_first})
                  n += 1

              #### 025 ####
              if (scenes[index][1]-scenes[index][0] >= 500) and (currentframe - 1 == idx_025):
                  filename_025 = "{}/{:0>6d}.jpg".format(folder_path, idx_025)
                  # video_save = cv2.resize(video[idx_025], (1280,720))
                  cv2.imwrite(filename_025, frame)

                  # SAVE CSV
                  writer.writerow({'n':n,'pts_time': (idx_025/25),'fps': 25,'frame_idx': idx_025})
                  n += 1

              # #### 05 ####
              if currentframe - 1 == idx_05:
                  filename_05 = "{}/{:0>6d}.jpg".format(folder_path, idx_05)
                  # video_save = cv2.resize(video[idx_05], (1280,720))
                  cv2.imwrite(filename_05, frame)

                  # SAVE CSV
                  writer.writerow({'n':n,'pts_time': (idx_05/25),'fps': 25,'frame_idx': idx_05})
                  n += 1

              # #### 075 ####
              if (scenes[index][1]-scenes[index][0] >= 500) and (currentframe - 1 == idx_075):
                  filename_075 = "{}/{:0>6d}.jpg".format(folder_path, idx_075)
                  # video_save = cv2.resize(video[idx_075], (1280,720))
                  cv2.imwrite(filename_075, frame)

                  # SAVE CSV
                  writer.writerow({'n':n,'pts_time': (idx_075/25),'fps': 25,'frame_idx': idx_075})
                  n += 1

              # #### End ####
              if currentframe - 1 == idx_end:
                  filename_end = "{}/{:0>6d}.jpg".format(folder_path, idx_end)
                  # video_save = cv2.resize(video[idx_end], (1280,720))
                  cv2.imwrite(filename_end, frame)

                  # SAVE CSV
                  writer.writerow({'n':n,'pts_time': (idx_end/25),'fps': 25,'frame_idx': idx_end})
                  n += 1

                  index += 1

          else:
              break


    cam.release()
    cv2.destroyAllWindows()

/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/Database-BTC/Videos_L02/video/L02_V001.mp4
/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/KeyFrames_AIO_Pending/KeyFrames_L02_Optimize2/L02_V001
/content/drive/MyDrive/Knowledge/AIO2023/Competitions/HCM AI/Database-BTC/Videos_L02/video/L02_V001.mp4
[TransNet] Processing video frames 30897/30897
